In [2]:
###加载数据集###
import pandas as pd
data=pd.read_csv('./family_data.csv',index_col='family_id')
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


In [6]:
###计算preference cost ；n 代表 家庭个数， choice 分配的方案
def get_penalty(n,choice):
    penalty=None
    if choice==0:
        penalty=0
    if choice==1:
        penalty=50
    if choice==2:
        penalty=50+9*n
    if choice==3:
        penalty=100+9*n   
    if choice==4:
        penalty=200+9*n  
    if choice==5:
        penalty=200+18*n   
    if choice==6:
        penalty=300+18*n      
    if choice==7:
        penalty=300+36*n  
    if choice==8:
        penalty=400+36*n   
    if choice==9:
        penalty=500+(199+36)*n  
    if choice>9:
        penalty=500+(398+36)*n    
    return penalty    

In [14]:
import numpy as np
N_DAYS=100 #访问的天数
N_FAMILY=5000 #访问的家庭数量
MIN_OCCUPANCY=125 #最小家庭承载量
MAX_OCCUPANCY=300 #最大家庭承载量

###计算preference cost 每个家庭 什么时间访问 所产生的penalty
pcost_mat=np.full(shape=(N_FAMILY,100),fill_value=999999)
for f in range(N_FAMILY):
    f_num=data.loc[f,'n_people']
#     print(f_num)
    ##初始化的pcost_mat 赋值 等于每个家庭的 other choice 下的成本
    pcost_mat[f,:]=get_penalty(f_num,11)
    for choice in range(10):
        temp=data.loc[f][choice]
        penalty=get_penalty(f_num,choice)
        pcost_mat[f,temp-1]=penalty
print(pcost_mat) 
print(pcost_mat.shape)

[[2236 2236 2236 ... 2236 2236 2236]
 [2236 2236 2236 ... 2236 2236 2236]
 [1802 1802 1802 ... 1802 1802    0]
 ...
 [3104 3104  616 ... 3104 3104 3104]
 [ 390 2670 2670 ... 2670 2670 2670]
 [2236 2236 2236 ... 2236 2236 2236]]
(5000, 100)


In [86]:
###计算accounting cost; 前一天的参观人数 当天的参观人数
acost_mat=np.zeros(shape=(MAX_OCCUPANCY,MAX_OCCUPANCY),dtype=np.float64)
for i in range(acost_mat.shape[0]):
    for j in range(acost_mat.shape[1]):
        diff=abs(i-j)
        acost_mat[i,j]=max(0,(i-125)/400*i**(0.5+diff/50.0))
acost_mat        


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.61426498e+15, 3.22526096e+15, 2.87812553e+15, ...,
        7.41048581e+00, 8.30427666e+00, 9.30586908e+00],
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        8.36716954e+00, 7.46610759e+00, 8.36716954e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        9.44825702e+00, 8.43020770e+00, 7.52185316e+00]])

In [20]:
###计算family size ；desired
FAMILY_SIZE=data['n_people'].values
FAMILY_SIZE
DESIRED=data.values[:,:-1]-1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]])

In [23]:
!pip install ortools


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.7 MB 41 kB/s eta 0:00:015   |████▎                           | 1.8 MB 39 kB/s eta 0:05:01     |█████████████████████████▋      | 11.0 MB 9.7 kB/s eta 0:04:41


In [47]:
from ortools.linear_solver import pywraplp
###使用LP对绝大多数家庭进行规划
def solveLP():
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x={} #family id 在第J天是否会参加
    candidates=[[] for x in range(N_DAYS)]
    for i in range(N_FAMILY):# family id 
        for j in DESIRED[i,:]:# family id's choice
#             print(j)
            candidates[j].append(i)
            x[i,j]=solver.BoolVar('x[%i,%i]'%(i,j))
    #每天参观的人数
    daily_occupancy=[solver.Sum([x[i,j]*FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]
#     print(daily_occupancy)
    
    #每个家庭 10个choice中出现的总次数
    family_presence=[solver.Sum(x[i,j] for j in DESIRED[i,:]) for i in range(N_FAMILY)]
    
    #目标函数 preference cost only
    preference_cost=solver.Sum([pcost_mat[i,j]* x[i,j]for i in range(N_FAMILY) for j in DESIRED[i,:]])
    
    #最小化目标函数
    solver.Minimize(preference_cost)
    
    #约束条件添加
    for j in range(N_DAYS-1):
        solver.Add(daily_occupancy[j]-daily_occupancy[j+1]<=25)
        solver.Add(daily_occupancy[j+1]-daily_occupancy[j]<=25)
        
    for i in range(N_FAMILY):
        solver.Add(family_presence[i]==1)
   
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >=MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j] <=MAX_OCCUPANCY)
    
    result=solver.Solve()
    temp=[(i,j,x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i,:] if x[i,j].solution_value()>0]
    
    df=pd.DataFrame(temp, columns=['family_id','day','result'])
    print( solver.Objective().Value())
    return df
#     print(x)  
 

In [48]:
%%time
result=solveLP()   

73702.31696428571
CPU times: user 21.7 s, sys: 82.4 ms, total: 21.8 s
Wall time: 28.2 s


In [50]:
#############已经安排上的家庭
assigned_df=result[result.result>0.999]
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5075,4995,15,1.0
5076,4996,87,1.0
5077,4997,31,1.0
5078,4998,91,1.0


In [51]:
#####没有安排的家庭
unassgined_df=result[(result.result<0.999)&(result.result>1-0.999)]
unassgined_df

,family_id,day,result
60,59,38,0.25
61,59,14,0.75
242,240,32,0.75
243,240,56,0.25
265,262,31,0.50
...,...,...,...
4990,4912,8,0.40
4992,4914,38,0.60
4993,4914,43,0.40
5040,4961,53,0.75


In [52]:
##计算每天访问的人数
assigned_df['family_size']=FAMILY_SIZE[assigned_df.family_id]
assigned_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5075,4995,15,1.0,4
5076,4996,87,1.0,2
5077,4997,31,1.0,6
5078,4998,91,1.0,5


In [54]:
##按照天数进行聚合
occupancy=assigned_df.groupby('day').family_size.sum().values
occupancy

array([285, 271, 294, 293, 263, 242, 223, 247, 273, 297, 288, 292, 275,
       250, 245, 272, 292, 292, 271, 248, 223, 244, 264, 291, 292, 296,
       273, 249, 234, 251, 278, 283, 252, 235, 205, 184, 202, 233, 253,
       226, 210, 183, 204, 229, 247, 281, 256, 223, 204, 198, 222, 248,
       256, 223, 208, 185, 173, 196, 219, 198, 174, 141, 124, 121, 149,
       170, 178, 160, 136, 123, 125, 135, 158, 185, 158, 134, 121, 125,
       142, 167, 186, 167, 138, 120, 128, 155, 174, 203, 177, 152, 130,
       122, 130, 158, 183, 158, 128, 125, 122, 124])

In [56]:
###每天的剩余量
min_occupancy=np.array([max(0,MIN_OCCUPANCY-x) for x in occupancy])
print(min_occupancy)
max_occupancy=np.array([MAX_OCCUPANCY-x for x in occupancy])
max_occupancy

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 0 0 0 0 0 2 0 0 0 0
 0 0 4 0 0 0 0 0 0 5 0 0 0 0 0 0 0 3 0 0 0 0 0 0 3 1]


array([ 15,  29,   6,   7,  37,  58,  77,  53,  27,   3,  12,   8,  25,
        50,  55,  28,   8,   8,  29,  52,  77,  56,  36,   9,   8,   4,
        27,  51,  66,  49,  22,  17,  48,  65,  95, 116,  98,  67,  47,
        74,  90, 117,  96,  71,  53,  19,  44,  77,  96, 102,  78,  52,
        44,  77,  92, 115, 127, 104,  81, 102, 126, 159, 176, 179, 151,
       130, 122, 140, 164, 177, 175, 165, 142, 115, 142, 166, 179, 175,
       158, 133, 114, 133, 162, 180, 172, 145, 126,  97, 123, 148, 170,
       178, 170, 142, 117, 142, 172, 175, 178, 176])

In [57]:
####使用整数规划对其他未安排家庭进行求解
def solveIP(families,min_occupancy,max_occupancy):
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    #需要被安排的家庭
    n_familes=len(families)
    
    x={} #family id 在第J天是否会参加
    candidates=[[] for x in range(N_DAYS)]
    for i in families:# family id 
        for j in DESIRED[i,:]:# family id's choice
#             print(j)
            candidates[j].append(i)
            x[i,j]=solver.BoolVar('x[%i,%i]'%(i,j))
    #每天参观的人数
    daily_occupancy=[solver.Sum([x[i,j]*FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]
#     print(daily_occupancy)
    
    #每个家庭 10个choice中出现的总次数
    family_presence=[solver.Sum(x[i,j] for j in DESIRED[i,:]) for i in families]
    
    #目标函数 preference cost only
    preference_cost=solver.Sum([pcost_mat[i,j]* x[i,j]for i in families for j in DESIRED[i,:]])
    
    #最小化目标函数
    solver.Minimize(preference_cost)
    
        #约束条件添加
        
    for i in range(n_familes):
        solver.Add(family_presence[i]==1)
   
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >=min_occupancy[j])
        solver.Add(daily_occupancy[j] <=max_occupancy[j])
    
    result=solver.Solve()
    temp=[(i,j) for i in families for j in DESIRED[i,:] if x[i,j].solution_value()>0]
    
    df=pd.DataFrame(temp, columns=['family_id','day'])
    print( solver.Objective().Value())
    return df


In [59]:
%%time
unassinged=unassgined_df.family_id.unique()
result=solveIP(unassinged,min_occupancy,max_occupancy)
result

1279.0
CPU times: user 106 ms, sys: 4.03 ms, total: 110 ms
Wall time: 227 ms


,family_id,day
0,59,38
1,240,32
2,262,31
3,488,39
4,500,26
...,...,...
64,4850,4
65,4886,98
66,4912,17
67,4914,38


In [89]:
####最终结果进行排序和输出
final_result=pd.concat((assigned_df[['family_id','day']],result)).sort_values('family_id')
final_result


,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5075,4995,15
5076,4996,87
5077,4997,31
5078,4998,91


In [96]:
!pip install numba

Defaulting to user installation because normal site-packages is not writeable


In [97]:
from numba import njit

In [98]:
###根据安排情况 计算cost
acost_mat=np.zeros(shape=(MAX_OCCUPANCY+1,MAX_OCCUPANCY+1),dtype=np.float64)
for i in range(acost_mat.shape[0]):
    for j in range(acost_mat.shape[1]):
        diff=abs(i-j)
        acost_mat[i,j]=max(0,(i-125)/400*i**(0.5+diff/50.0))
acost_mat 

# preference cost
@njit(fastmath=True)
def pcost(prediction):
    daily_occupancy=np.zeros(N_DAYS+1, dtype=np.int64)
    penalty=0
    for (i,p) in enumerate(prediction):
        n=FAMILY_SIZE[i]
        penalty+=pcost_mat[i,p]
        daily_occupancy[p]+=n
    return penalty, daily_occupancy 
# accounting cost
@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost=0
    num_out_of_range=0
    daily_occupancy[-1]=daily_occupancy[-2]
    for day in range(N_DAYS):
        n_p1=daily_occupancy[day+1]
        n=daily_occupancy[day]
        num_out_of_range+=(n>MAX_OCCUPANCY) or (n<MIN_OCCUPANCY)
        accounting_cost+=acost_mat[n,n_p1]
    return  accounting_cost, num_out_of_range 
##total cost
@njit(fastmath=True)
def cost_function(prediction):
    penalty, daily_occupancy=pcost(prediction)
    accounting_cost, num_out_of_range =acost(daily_occupancy)
    final_score=penalty+accounting_cost+num_out_of_range*99999999
    return final_score

In [91]:
prediction=final_result.day.values
prediction   

array([51, 25, 99, ..., 31, 91, 12])

In [77]:
print(cost_function(prediction))

77480.2439961357


In [81]:
########################最终安排的file：days+1 ###############################
final_results=final_result
final_results['assigned_day']=final_results['day']+1
final_results.drop(['day'],axis=1,inplace=True)
final_results


,family_id,assigned_day
0,0,52
1,1,26
2,2,100
3,3,2
4,4,53
...,...,...
5075,4995,16
5076,4996,88
5077,4997,32
5078,4998,92


In [100]:
#############对原方案进行优化################
def find_better(pred):
    fobs=np.argsort(FAMILY_SIZE)
    
    score=cost_function(pred)
    original_score=np.inf
    
    while score<original_score:
        original_score=score
        
        for family_id in fobs:
            for pick in range(10):
                day=DESIRED[family_id,pick]
                oldvalue=pred[family_id]
                pred[family_id]=day
                
                new_score=cost_function(pred)
                
                if new_score<score:
                    score=new_score
                else:
                    pred[family_id]=oldvalue
                    
        print(score ,end='\r')    
    print(score)


In [99]:
new=prediction.copy()
new


array([51, 25, 99, ..., 31, 91, 12])

In [101]:
find_better(new)

73751.58816071771
